In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

%matplotlib inline
sns.set(style="whitegrid")

1.2. Анализ и очистка набора данных



In [ ]:
df.describe()

In [ ]:
# Удалим timestamp (не нужен для базовой модели)
df = df.drop('timestamp', axis=1)
df.head()

**2.2. Формирование обучающей и тестирующей выборок**

In [ ]:
# Разбиение 75% train / 25% test
trainset, testset = train_test_split(data, test_size=0.25)

print(f"Обучающая выборка: {len(trainset.all_ratings())} рейтингов")
print(f"Тестовая выборка: {len(testset)} рейтингов")

2.3. Обучение моделей и предсказания



In [ ]:
# Обучение базовых моделей
svd.fit(trainset)
knn.fit(trainset)
slope.fit(trainset)
cocluster.fit(trainset)

# Предсказания на тесте
pred_svd = svd.test(testset)
pred_knn = knn.test(testset)
pred_slope = slope.test(testset)
pred_cocluster = cocluster.test(testset)

# Простой ансамбль: усреднение предсказаний
def ensemble_predict(testset):
    preds = []
    for uid, iid, true_r in testset:
        est = (svd.predict(uid, iid).est +
               knn.predict(uid, iid).est +
               slope.predict(uid, iid).est +
               cocluster.predict(uid, iid).est) / 4
        preds.append((uid, iid, true_r, est))
    return preds

pred_ensemble = ensemble_predict(testset)

# Визуализация распределения предсказанных рейтингов (ансамбль)
estimates = [p[3] for p in pred_ensemble]
plt.figure(figsize=(8,5))
sns.histplot(estimates, bins=20, kde=True)
plt.title('Распределение предсказанных рейтингов (ансамбль)')
plt.xlabel('Предсказанный рейтинг')
plt.show()

3.1. Расчёт эффективности



In [ ]:
# Метрики для отдельных моделей
print("SVD:")
accuracy.rmse(pred_svd, verbose=False)
accuracy.mae(pred_svd, verbose=False)

print("KNN:")
accuracy.rmse(pred_knn, verbose=False)
accuracy.mae(pred_knn, verbose=False)

print("SlopeOne:")
accuracy.rmse(pred_slope, verbose=False)
accuracy.mae(pred_slope, verbose=False)

print("CoClustering:")
accuracy.rmse(pred_cocluster, verbose=False)
accuracy.mae(pred_cocluster, verbose=False)

# Метрики для ансамбля
def calculate_metrics(predictions):
    rmse = np.sqrt(np.mean([(true - est)**2 for (_, _, true, est) in predictions]))
    mae = np.mean([abs(true - est) for (_, _, true, est) in predictions])
    return rmse, mae

rmse_ens, mae_ens = calculate_metrics(pred_ensemble)
print(f"Ensemble RMSE: {rmse_ens:.4f}")
print(f"Ensemble MAE: {mae_ens:.4f}")

# Сводная таблица
results = pd.DataFrame({
    'Model': ['SVD', 'KNN', 'SlopeOne', 'CoClustering', 'Ensemble'],
    'RMSE': [accuracy.rmse(pred_svd, verbose=False),
             accuracy.rmse(pred_knn, verbose=False),
             accuracy.rmse(pred_slope, verbose=False),
             accuracy.rmse(pred_cocluster, verbose=False),
             rmse_ens],
    'MAE': [accuracy.mae(pred_svd, verbose=False),
            accuracy.mae(pred_knn, verbose=False),
            accuracy.mae(pred_slope, verbose=False),
            accuracy.mae(pred_cocluster, verbose=False),
            mae_ens]
})
results

3.2. Оптимизация моделей



In [ ]:
from surprise.model_selection import GridSearchCV

# GridSearch для SVD
param_grid = {'n_factors': [50, 100], 'n_epochs': [20, 30], 'lr_all': [0.005, 0.01]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print("Лучшие параметры SVD:", gs.best_params['rmse'])
print("Лучший RMSE:", gs.best_score['rmse'])

# Обучим оптимизированный SVD
best_svd = gs.best_estimator['rmse']
best_svd.fit(trainset)
pred_best_svd = best_svd.test(testset)

# Новый ансамбль с оптимизированным SVD
def ensemble_optimized(testset):
    preds = []
    for uid, iid, true_r in testset:
        est = (best_svd.predict(uid, iid).est +
               knn.predict(uid, iid).est +
               slope.predict(uid, iid).est +
               cocluster.predict(uid, iid).est) / 4
        preds.append((uid, iid, true_r, est))
    return preds

pred_ens_opt = ensemble_optimized(testset)
rmse_opt, mae_opt = calculate_metrics(pred_ens_opt)
print(f"Оптимизированный Ensemble RMSE: {rmse_opt:.4f}")

# Визуализация сравнения
models_comp = ['Baseline Ensemble', 'Optimized Ensemble']
rmse_comp = [rmse_ens, rmse_opt]
plt.bar(models_comp, rmse_comp)
plt.title('Сравнение RMSE до и после оптимизации')
plt.ylabel('RMSE')
plt.show()